# NLP with CNN

### Exercise objectives:

- Use CNN instead of RNN for NLP

<hr>
<hr>


# The data


❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy.

In [1]:
######################################
### Run this cell to load the data ###
######################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2022-11-18 12:27:29.910266: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-18 12:27:35.553238: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Remember that to do NLP, you need to go through one of the following options, as shown here: 

<img src="embedding_or_RNN.png" width="700px" />

But in both cases, you can replace the recurrent layer (top part) by a CNN layer. We will go into both, starting from the one on the left were the embedding is learned in the network.




# Part 1 : Concatenate a Keras Embedding with a Conv1D🔥 

Let's train a fancy network here. 

Each of your words is represented by a vector of size N (the size of your embedding). Therefore, as a sentence is a sequence of words, it is represented by a matrix (number of words, N). So, all your sentences are actually represented as matrices once embedded.

If you think about it, an image is also a matrix. Said differently, you may represent your sentence of word as a matrix, where each column (or row, depending on how you want to look at it) is a word, and each row (or each column) corresponds to a coordinate in the embedding space. As shown here

<img src="image_comparison.png" width="500px" />

Well, in that case, as these are close to images, why not using convolution on them? Yes, convolutions!
But, be careful. In the case of images, convolutions are 2 dimensional as the filters can move up and down, and left and right. In the case of our sentences, we want the kernel to move _only_ in the word by word direction (The alternative, moving coordinate by coordinate of the embedding space doesn't make much sense).

So let's create a model that use convolutions.

## First, the data

❓ **Question** ❓ You will need to prepare your data. First, tokenize them. Then, you need to pad them (use a value `maxlen` equal to 150). You also might need to compute the size of your vocabulary ;)

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer

# This initializes a Keras utilities that does all the tokenization for you
tokenizer = Tokenizer()

# The tokenization learns a dictionary that maps a token (integer) to each word
# It can be done only on the train set - we are not supposed to know the test set!
# This tokenization also lowercases your words, apply some filters, and so on - you can check the doc if you want
tokenizer.fit_on_texts(X_train)
    
# We apply the tokenization to the train and test set
X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)



vocab_size = len(tokenizer.word_index)#len(set([item for sublist in X_train_token for item in sublist]))
vocab_size

30419

In [3]:
from tensorflow.keras.utils import pad_sequences

X_train_pad = pad_sequences(
    X_train_token,
    maxlen=150,
    dtype=type(X_train_token[0][0]),
    padding='post',
    value=0
)


X_test_pad = pad_sequences(
    X_test_token,
    maxlen=150,
    dtype=type(X_train_token[0][0]),
    padding='post',
    value=0
)
len(X_train_pad[0]),len(X_test_pad[0])

(150, 150)

## Using 1D Convolution.

❓ **Question** ❓ Define a model that has :
- an `Embedding` layer: `input_dim` is the `vocab_size + 1`, `output_dim` is the embedding space dimension, and `mask_zero` has to be set to `True`. Here, for computational reasons, set `input_length` to the maximum length of your observations (that you just defined in the previous question).
- a `Conv1D` layer 
- a `Flatten` layer
- a `Dense` layer
- an output layer

Compile the model accordingly

❗ **Remark** ❗ The size of the `Conv1D` kernel corresponds exactly to the number of side-by-side words (tokens) each kernel is taking into account ;)

In [4]:
### Let's build the neural network now
from tensorflow.keras import layers, Sequential

# Size of your embedding space = size of the vector representing each word
embedding_size = 50

model_cnn = Sequential()
model_cnn.add(layers.Embedding(
    input_dim=vocab_size+1, # 16 +1 for the 0 padding
    input_length=150, # Max_sentence_length (optional, for model summary)
    output_dim=embedding_size, # 50
    mask_zero=True, # Built-in masking layer :)
))

model_cnn.add(layers.Conv1D(20,kernel_size=3))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(20))
model_cnn.add(layers.Dense(1, activation="sigmoid"))

model_cnn.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

❓ **Question** ❓ Look at the number of parameters. You can compare it to the model that you had in previous exercise (esp. the first one)

In [5]:
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 50)           1521000   
                                                                 
 conv1d (Conv1D)             (None, 148, 20)           3020      
                                                                 
 flatten (Flatten)           (None, 2960)              0         
                                                                 
 dense (Dense)               (None, 20)                59220     
                                                                 
 dense_1 (Dense)             (None, 1)                 21        
                                                                 
Total params: 1,583,261
Trainable params: 1,583,261
Non-trainable params: 0
_________________________________________________________________


❓ **Question** ❓ Fit your model with a stopping criterion, and evaluate it on the test data.

You will probably notice that it is ... **much faster** than RNN.

In [6]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model_cnn.fit(X_train_pad, y_train, 
          epochs=20, 
          batch_size=32,
          validation_split=0.3,
          callbacks=[es]
         )


res = model_cnn.evaluate(X_test_pad, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

Epoch 1/20
55/55 [==============================] - 2s 12ms/step - loss: 0.6860 - accuracy: 0.5389 - val_loss: 0.6512 - val_accuracy: 0.6453
Epoch 2/20
55/55 [==============================] - 0s 8ms/step - loss: 0.3378 - accuracy: 0.9286 - val_loss: 0.5180 - val_accuracy: 0.7267
Epoch 3/20
55/55 [==============================] - 0s 8ms/step - loss: 0.0583 - accuracy: 0.9909 - val_loss: 0.6212 - val_accuracy: 0.7280
Epoch 4/20
55/55 [==============================] - 0s 8ms/step - loss: 0.0058 - accuracy: 1.0000 - val_loss: 0.5328 - val_accuracy: 0.7827
Epoch 5/20
55/55 [==============================] - 0s 7ms/step - loss: 9.4409e-04 - accuracy: 1.0000 - val_loss: 0.5990 - val_accuracy: 0.7707
Epoch 6/20
55/55 [==============================] - 0s 8ms/step - loss: 5.0039e-05 - accuracy: 1.0000 - val_loss: 0.6509 - val_accuracy: 0.7720
Epoch 7/20
55/55 [==============================] - 0s 8ms/step - loss: 6.4666e-05 - accuracy: 1.0000 - val_loss: 0.7613 - val_accuracy: 0.7560
The acc

# Part 2 : Learn a Word2Vec representation, and then feed it into a NN with a `Conv1D`🔥 

In the first part of the exercise, you were asked to jointly learn the embedding representation and the CNN convolution within the same network, which was the CNN counterpart of the left part of this Figure: 

<img src="embedding_or_RNN.png" width="700px" />

Now, let's try to replace the RNN with a CNN for an architecture as on the right side.

❓ **Question** ❓ Learn a word2vec model or load a trained one directly from GENSIM (transfer learning). Then, prepare your data as in the previous exercise. This question is quite long, it prepares you for real world challenges - but you have already done all the building bricks in the previous exercises

In [7]:
import gensim.downloader as api
import numpy as np
word2vec_transfer = api.load("glove-wiki-gigaword-50")

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

# Pad the training and test embedded sentences
X_train_pad_2 = pad_sequences(X_train_embed_2, dtype='float32', padding='post', maxlen=150)
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype='float32', padding='post', maxlen=150)

❓ **Question** ❓ Now construct a model that has a `Conv1D` layer, a flatten layer, a dense layer, and an output layer. Compile it, and fit it on the train data. You can then evaluate it on the test set.

In [15]:
X_train_pad_2[0]

array([[ 0.53074 ,  0.40117 , -0.40785 , ...,  0.28762 ,  0.1444  ,
         0.23611 ],
       [ 0.086888, -0.19416 , -0.24267 , ..., -0.77    ,  0.3945  ,
        -0.16937 ],
       [ 0.36143 ,  0.58615 , -0.23718 , ...,  0.39362 ,  0.36523 ,
         0.36077 ],
       ...,
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ]], dtype=float32)

In [16]:
model_tf = Sequential()

# model_tf.add(layers.Masking(mask_value=0, input_shape=X_train_pad_2.shape[1:]))
model_tf.add(layers.Conv1D(20,kernel_size=3))
model_tf.add(layers.Flatten())
model_tf.add(layers.Dense(20))
model_tf.add(layers.Dense(1, activation="sigmoid"))

model_tf.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [17]:
es = EarlyStopping(patience=5, restore_best_weights=True)

model_tf.fit(X_train_pad_2, y_train, 
          epochs=20, 
          batch_size=32,
          validation_split=0.3,
          callbacks=[es]
         )


res = model_tf.evaluate(X_test_pad_2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

Epoch 1/20
55/55 [==============================] - 1s 10ms/step - loss: 0.9795 - accuracy: 0.5429 - val_loss: 0.7076 - val_accuracy: 0.6093
Epoch 2/20
55/55 [==============================] - 0s 6ms/step - loss: 0.5488 - accuracy: 0.7383 - val_loss: 0.6858 - val_accuracy: 0.6240
Epoch 3/20
55/55 [==============================] - 0s 5ms/step - loss: 0.4016 - accuracy: 0.8234 - val_loss: 0.7400 - val_accuracy: 0.6507
Epoch 4/20
55/55 [==============================] - 0s 6ms/step - loss: 0.2924 - accuracy: 0.8857 - val_loss: 0.8078 - val_accuracy: 0.6400
Epoch 5/20
55/55 [==============================] - 0s 6ms/step - loss: 0.2095 - accuracy: 0.9269 - val_loss: 0.9702 - val_accuracy: 0.6307
Epoch 6/20
55/55 [==============================] - 0s 6ms/step - loss: 0.1256 - accuracy: 0.9623 - val_loss: 1.1665 - val_accuracy: 0.6187
Epoch 7/20
55/55 [==============================] - 0s 5ms/step - loss: 0.0832 - accuracy: 0.9794 - val_loss: 1.4972 - val_accuracy: 0.6160
The accuracy evalua

❓ **Question** ❓ You might be frustrated by the accuracy you got, this happens to us all at some point. Once you have tested your first iteration, you need to improve your models: by making them more complex, changing the parameters, stacking additional layers, and so on.

Only practice and experimentation will get you there. So you can go back to your previous models, change them and try to get better results ;)